In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
"""
선형 모델 : outlier를 하나하나씩 잡는게 중요하다.
어떻게 찾고, 어떻게 지울 것인가, 또 어떤거는 지우면 안 될 것인가.
"""

## 데이터 불러오고 합치기 ##

In [ ]:
train = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv') 
test =  pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv')

In [ ]:
## outlier 제거 ##

In [ ]:
#대괄호->두가지 옵션 가능. 
#1) 칼럼 넣기
#2) 조건을 받을 수 있음!!!! 그래서 행이 들어가지는 것 (masking이라고 부름)

train = train.drop(train[(train['GrLivArea']>=4000) & (train['SalePrice']<=200000)].index) # &는 비교 연산도 되나봄

In [ ]:
train = train.drop(train[(train['GarageArea']>=1230) & (train['SalePrice']<=300000)].index)

In [ ]:
## 데이터 보기 ##

In [ ]:
pd.options.display.max_columns = 500

In [ ]:
display(train,test)
#결과 : column 1개 차이 SalePrice
# 결측치, 문자값(범주형), 칼럼많음
# 전처리 너무 많음. -> train,test를 alldata로 묶어버린 다음에, 한 번에 전처리 -> train,test로 나눠

In [ ]:
## train,test 합쳐서 전처리위해 alldata만들기 ##

In [ ]:
alldata = pd.concat([train,test],axis=0) #axis=0은 기본값. 0이 행으로 합침
alldata
#alldata로 해주는 이유 -> 전처리를 한 번에 쉽게 해주기 위해!! 되도록 이렇게 해주는게 좋음!!

In [ ]:
#### 통계량추가 ####

In [ ]:
LotFrontage_mean = alldata.groupby('Neighborhood')['LotFrontage'].mean().to_frame('LotFrontage_mean').reset_index()  #output = pd
# groupby ()안에 들어있는 Neighborhood의 값들(다른)을 기준으로 묶어서 봄 -> 
# to_frame : 칼럼 이름 바꾸기

In [ ]:
alldata = pd.merge(alldata,LotFrontage_mean,on='Neighborhood',how='left')
# on : 어떤 칼럼을 기준으로 묶을래
# how : 어디다 붙일래 left=alldata에 붙이겠다.


In [ ]:
alldata

In [ ]:
#### 교호작용(데이터 전처리 중 2개의 칼럼 정보 합치기)####

In [ ]:
alldata['old'] = alldata['YrSold']-alldata['YearBuilt'] #그냥 빼기하면, 원소별 빼기 가능

In [ ]:
alldata['totalArea'] = alldata['GrLivArea'] + alldata['TotalBsmtSF']

In [ ]:
alldata['finalArea'] = alldata['totalArea'] + alldata['GarageArea']

In [ ]:
alldata['Area'] = alldata['finalArea'] + (alldata['LotArea']/8) #10으로 나눠주는게 더 점수 좋아지네. 스케일이 더 맞춰지나보다.

In [ ]:
## 통계량 추가 ##

In [ ]:
Area_mean = alldata.groupby('Neighborhood')['Area'].mean().to_frame('Area_mean').reset_index()
alldata = pd.merge(alldata,Area_mean,on='Neighborhood',how='left')

In [ ]:
alldata

In [ ]:
alldata['Qual_Cond_plus'] = alldata['OverallQual'] + 0.4*alldata['OverallCond']  # 10:4 비율로 가중치 주겠다.

In [ ]:
alldata['Qual_Cond_minus'] = alldata['OverallQual'] - 0.4*alldata['OverallCond']

In [ ]:
## 통계량 추가 ##
Qual_Cond_plus_mean = alldata.groupby('Neighborhood')['OverallQual'].mean().to_frame('Qual_Cond_plus_mean').reset_index()
alldata = pd.merge(alldata,Qual_Cond_plus_mean,on='Neighborhood',how='left')
alldata

In [ ]:
## 데이터 전처리 ##

In [ ]:
#Neighborhood 칼럼 추가(label encoder형태로)  -> 잘못된 방법
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder() #클래스임

# 1)등록하기(fit) 2)transform(문자->숫자)
# alldata['Neighborhood'] = le.fit_transform(alldata['Neighborhood'])

In [ ]:
#Neighborhood 원핫인코딩으로 해야함! (선형모델이라)
# pd.get_dummies(alldata['Neighborhood']) 
# #concat으로 열방향으로 합친다.
# alldata = pd.concat([alldata,pd.get_dummies(alldata['Neighborhood'])],axis=1) #원핫인코딩된 칼럼을 추가시킴

In [ ]:
"""
위 코드 문제 : 모든 문자형 칼럼을 이렇게 처리하기 어렵다->모델이 알아서 중요한 칼럼을 선택해주고, 해줬으면 좋겠다.
"""

In [ ]:
#alldata['MoSold2'] = alldata['MoSold']

In [ ]:
#### 원 핫 인코딩 ####

In [ ]:
#type바꾸기
alldata['MSSubClass']= alldata['MSSubClass'].astype('object')

In [ ]:
alldata = pd.get_dummies(alldata) #알아서 문자형 칼럼을 찾아 원핫인코딩 해줌

In [ ]:
alldata #이런식으로 추가된다, 원핫인코딩 하면.

# 모든 문자열 칼럼을 원핫인코딩해서 학습시켰는데, 그렇게 성능이 좋아지지 않았따. 중요한 칼럼만 선택해야한다.
# 모델에서 규제필요 <-특히, 과적합이 일어날 수 있는 확률이 높은 칼럼한테(특정카테고리에 데이터가 몰려있는 경우에)
# but, 모델에서의 규제는, 하기 전에 scaling이 필요(값이 큰 것은, 조금만 규제해도 확 값이 달라지고, 값이 작은 걸 규제하면, 변화가 미미하기 때문) ->alldata2에서

In [ ]:
#### 데이터 확인(matplotlib, sns) ####

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

## boxplot

plt.figure(figsize=(20,11))
sns.boxplot(train['Neighborhood'],train['SalePrice'])
#Neighborhood
#도움이 됨 -> value에 따라 그래프가 아주 다름

In [ ]:
plt.figure(figsize=(20,11))
sns.stripplot(train['Neighborhood'],train['SalePrice'])
#box plot [x:범주형 y:선형] . stripplot은 boxplot을 좀더 자세히 볼 수 있는 방법

In [ ]:
plt.figure(figsize=(20,11))
sns.swarmplot(train['Neighborhood'],train['SalePrice'])
#boxplot->stripplot->swarmplot :데이터가 많은 때, swarmplot으로 보면, 더 잘 볼 수 있다.

In [ ]:
train['Neighborhood'].value_counts()
#데이터가 적은 것 같지만, 원래 데이터셋의 수가 적어서 그런것 -> 비율로 봐야함. value가 여러개이고, 꽤 많이 있으니 도움됨
#NPkVill,Blueste -> 데이터 너무 적음-> 과적합될 수 있음

In [ ]:
test['Neighborhood'].value_counts()

In [ ]:
#선형-선형 -> scatter plot쓰자.
plt.figure(figsize=(20,12))
sns.scatterplot(train['OverallQual'],train['SalePrice']) #correaltion 높아서, 우상향 느낌 남

In [ ]:
plt.figure(figsize=(20,12))
sns.scatterplot(train['GrLivArea'],train['SalePrice'])

In [ ]:
## outlier제거(GRLiveArea, OverallQual고려)
(train['GrLivArea']>=4000) & (train['SalePrice']<=200000)

In [ ]:
# 집이 오래될 수록 값이 싸지는 우하향 그래프
plt.figure(figsize=(20,12))
sns.scatterplot(alldata['old'],alldata['SalePrice'])

In [ ]:
plt.figure(figsize=(20,12))
sns.scatterplot(alldata['totalArea'],alldata['SalePrice'])

In [ ]:
alldata.corr()['SalePrice']['totalArea']

In [ ]:
plt.figure(figsize=(20,12))
sns.scatterplot(alldata['finalArea'],alldata['SalePrice'])

In [ ]:
alldata.corr()['SalePrice']['finalArea'] # correlation이 0.85? 상관관계가 엄청나네. 중요한 칼럼이다.

In [ ]:
#행접근
#train.loc[0] 행의 이름으로 접근#,train.iloc[0] 행의 순서

plt.figure(figsize=(20,12))
sns.distplot(train['SalePrice']) #가격의 갯수에 따른 분포(얼마에 몇개가?)

In [ ]:
plt.figure(figsize=(20,12))
sns.scatterplot(train['GarageArea'],train['SalePrice'])

In [ ]:
len(train)

In [ ]:
len(test)

In [ ]:
#### 데이터 전처리 ####

In [ ]:
"""
어떤 모델을 선택?
이전 : RF 범주형데이터가 많아서
지금 데이터 : 숫자적인 느낌의 데이터가 굉장히 많다.
숫자형데이터가 많은가? correlation보기 (x,y)간의 선형성 판단 (correlation함수 이용해서)
"""

In [ ]:
#### correlation 확인 ####

In [ ]:
alldata.corr() #각 칼럼들과의 correlation계산

In [ ]:
#상관관계 파악 
alldata.corr()['SalePrice'].sort_values(ascending=False)[:30] #'SalesPrice'와의 선형성 판단해서 수치형데이턴지 확인
# 0.3보다 크면, 선형성이 강함 -> 트리들이 학습하기 어려움 ->선형모델써야함(SVM,딥러닝...)


In [ ]:
alldata.corr()['SalePrice'].sort_values(ascending=False) # -방향으로 중요하다? 의미 : old가 커질수록,saleprice는 하향을 의미 (corr값이 0.5니까, 선형성은 가짐)

In [ ]:
"""
1.범주형데이터 -> 원핫인코딩
2.scaling : [0,1]로.  정규화랑(평균=0, 분산=1) 조금 다름
3.결측치(NAn) 처리 + 필요없는 칼럼 제거
4.TRAIN,TEST칼럼 개수 맞추기
"""

In [ ]:
"""

"""

In [ ]:
#### 문자(object)칼럼 삭제 ####

In [ ]:
#문자 데이터 ㅅ칼럼 삭제
# baseline : 문자 말고 숫자로만 학습
alldata= alldata[alldata.columns[alldata.dtypes!='object']]
alldata

In [ ]:
#### 결측치 채우기 ####

In [ ]:
#결측치 채우기
alldata.isna() 

In [ ]:
alldata.isna().sum() #결측치 NaN확인

In [ ]:
#결측치 채우는 방법
"""
1. 칼럼 기존의 안의 내용과 다른 느낌으로 채움 (칼럼 안에 들어가 있지 않은 값으로 채움)
ex. -1로 채움
"""

In [ ]:
alldata = alldata.fillna(-1)
# fillna
alldata.isna().sum()

In [ ]:
#### 필요없는 칼럼 드랍 시키기 ####

In [ ]:
# 필요없는 칼럼, train,test칼럼개수 맞추기
alldata2 = alldata.drop(['Id','SalePrice'],axis=1)

In [ ]:
alldata2.columns #칼럼 확인

In [ ]:
#### scaling 하기 ####

In [ ]:
# scaling
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
# ss = StandardScaler()
rs = RobustScaler()
#alldata2 = ss.fit_transform(alldata2) #array형식으로 바뀜
alldata2 = rs.fit_transform(alldata2) 
# class를 dataframe에 적용시키는 방법<-fit_transform
# 중요! : 원핫인코딩 먼저하고, scaling해야한다!!!(원핫인코딩한 것도, 정규화해줘야하기때문)

In [ ]:
alldata2

In [ ]:
alldata2.shape  

In [ ]:
alldata2[0]

In [ ]:
#### train, test 데이터 쪼개기 (alldata로부터)

In [ ]:
train2 = alldata2[:len(train)] #alldata에서 train의 개수만큼만 가져오겠다.
test2 = alldata2[len(train):] #alldata에서 test의 데이터개수만큼 가져오겠다.
#alldata : train,test를 합친 것

In [ ]:
alldata

In [ ]:
#alldata.dtypes!='object'

In [ ]:
# from sklearn.model_selection import train_test_split
# x_train,x_valid,y_train,y_valid = train_test_split(train2,train['SalePrice'],test_size=0.2,random_state=777) # 

In [ ]:
#### 학습 ####
# 선형모델 이용 (칼럼 많고, 칼럼 보니까 연속값 많음)
# cross_val로 교차검증 수행 

In [ ]:
# 데이터가 적기 때문에 교차검증(=k fold 검증) 필요 ->효과 : 모든 데이터로 학습, 모든 데이터로 검증 (k번 검증한 것 평균 냄.)
"""
구글,교차검증 이미지 검색.
"""
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
for i in [7,8,9,10,11,12,13,14] :
    #[10,20,50,100,200,300,500,700,1000
    ridge = Ridge(alpha = i)

    print(np.sqrt(-cross_val_score(ridge,train2,train['SalePrice'],n_jobs=-1,cv=10,scoring='neg_mean_squared_error').mean()))
# parameter : 모델, x, y, n_jobs, 등분수(k), 평가지표 neg붙이는 이유 : sckitlearn에서 최적의 모델을 선택할 때, 값이 무조건(평가지표가) 큰 것을 선택하는 데, error를 평가지표로 사용하면 이게 안 좋은 것이기 때문.
# r결과 : 13이 괜찮겠다.

In [ ]:
from sklearn.linear_model import Ridge
# 선형모델 : Ridge 라쏘 (?) 엘라스틴넷(?)
# 선형 모델 -> 어떤 칼럼이 더 중요한지 잘 모름->가중치둬서 학습시켜줘야함
ridge = Ridge(alpha = 13) #alpha값이 커질수록, 규제가 커짐. 칼럼많으면, 규제많이 주는게 좋음
#최적의 alpha값 ->내부적으로 찾자. ->평가셋가지고 보면서. -> 그리드서치

ridge.fit(train2,np.log(train['SalePrice']))  #gradient model만, 학습하면서 평가가능!! 다른 모델은 학습다시키고, 예측한후 평가해야함

result = ridge.predict(test2)

In [ ]:
## 트리모델 테스트 ##
from catboost import CatBoostRegressor
#트리모델, 범주형 데이터 잘 분류해줌.
cat =  CatBoostRegressor(iterations=4000) #iterations에 맞게 lr이 자동으로 조절되는게 catboost의 장점
cat.fit(train2,np.log(train['SalePrice']))
result2 = cat.predict(test2)

In [ ]:
# from sklearn.linear_model import Ridge
# # 선형모델 : Ridge 라쏘 (?)
# # 선형 모델 -> 어떤 칼럼이 더 중요한지 잘 모름->가중치둬서 학습시켜줘야함
# ridge = Ridge(alpha = 10) #alpha값이 커질수록, 규제가 커짐. 칼럼많으면, 규제많이 주는게 좋음
# #최적의 alpha값 ->내부적으로 찾자. ->평가셋가지고 보면서.

# ridge.fit(train2,train['SalePrice'])

# result = ridge.predict(test2)

In [ ]:
#### 평가(metrics) 및 test #####

In [ ]:
from sklearn.metrics import mean_squared_error

#np.sqrt(mean_squared_error(y_valid,result)) #왼쪽을 정답값 쓰는 습관을 들이자.

In [ ]:
result

In [ ]:
#### 제출 ####

In [ ]:
csv = pd.read_csv('/kaggle/input/home-data-for-ml-course/sample_submission.csv')
csv

In [ ]:
len(csv)

In [ ]:
len(result)

In [ ]:
#csv['SalePrice'] = np.exp(result)
#csv['SalePrice'] = np.exp(result2)
#### 앙상블 제출 ####
csv['SalePrice'] = 0.47*np.exp(result)+0.53*np.exp(result2)

In [ ]:
csv.to_csv('HousingPrice_baseline.csv',index=0) #인덱스는 빼겠다.